In [1]:
# importing all the libraries
import xml.etree.ElementTree as ET
import string
import os  
import re

In [2]:
# function to remove html tags from the text and everything inside the tags.
def cleanhtml(raw_html):
  cleanr = re.compile('<.*?>')
  cleantext = re.sub("[^a-zA-Z]", " ", str(raw_html))

  return cleantext

In [3]:
# Function to clean text and output it to a text file in comma seperated format
fulltext = "fulltext"
xml =".xml"

def getcsv(outfile, parent_dir, word):

    for root, dirs, files in os.walk(parent_dir): # To search through the directory in aloop
        for name in files:
            if name =='fulltext.xml': # finding fulltext.xml file within the directory
                new_path = (os.path.join(root, name))
                file_name = os.path.dirname(new_path) # getting the directory path as a variable
                tree = ET.parse(new_path)
                root = tree.getroot()
                abstracts = root.findall("./front/article-meta/abstract/") # searching the fulltext file to get abstract

                for abstract in abstracts:
                    abstractText = (abstract.text)
                    abstractNoTags= cleanhtml(abstractText) # applying the cleantext function 
                    abstractNoTagsLower = abstractNoTags.lower() # converting the entire text to lower case
                    input_str = re.sub(r'\d+', '', abstractNoTagsLower)  #Regular Expression, or regex 
                    #input_str = input_str.translate(string.maketrans("",""), string.punctuation)
                    new_1 = input_str.strip() #Remove spaces at the beginning and at the end of the string
                    csvLine = str(file_name) +  "," + new1 +  "," + word  + "\n"
                    outp = csvLine  + "\n" # put output in a string

                    outfile.write(outp) # write to a text file

    outfile.close() #close the text file

In [4]:
# using the function getcsv for true positives
outfile1 = open('true.txt','w')
parent_dir_true  = "D:\\try1\\true"
word1 = "true"

getcsv(outfile1, parent_dir_true, word1)

# using the function getcsv for false positives
outfile2 = open('false.txt','w')
parent_dir_false  = "D:\\try1\\false"
word2 = "false"

getcsv(outfile2, parent_dir_false, word2)

In [5]:
# function removing words like abstract etc.
def remove_abs(out_put, in_put):
    with open(in_put, "r") as a_file: #open the file
      for line in a_file: #loop through each word in the file
        if len(line.split()) >2: # if the word length for the line is greater than 2
            out_put.write(line) # write to a new file

    out_put.close()

D:\try1


In [6]:
# using the function remove_abs
in_put1 = 'true.txt'
in_put2 = 'false.txt'

out_put1 = open('true_new.txt','w')
out_put2 = open('false_new.txt','w')

remove_abs(out_put1,in_put1)
remove_abs(out_put2, in_put2)

In [7]:
# merging two files (false positives and true positives)
  
data = data2 = "" 
  
# Reading data from file1 
with open('true_new.txt') as fp: 
    data = fp.read() 
  
# Reading data from file2 
with open('false_new.txt') as fp: 
    data2 = fp.read() 
  
# Merging 2 files 
# To add the data of file2 
# from next line 
data += "\n"
data += data2 
  
with open ('OpenVirus.txt', 'w') as fp: 
    fp.write(data) 

In [8]:
# converting the text file into csv file
import pandas as pd

read_file = pd.read_csv (r'OpenVirus.txt') #read the file as a pandas dataframe
read_file.to_csv (r'OpenVirus.csv', index=None) #write it to csv file

In [12]:
#adding headers to the file 

import csv
with open('OpenVirus.csv',newline='') as f:
    r = csv.reader(f)
    data = [line for line in r]
with open('OpenVirus.csv','w',newline='') as f:
    w = csv.writer(f)
    w.writerow(['ArticleId','Text', 'Category'])
    w.writerows(data)

In [13]:
df_train = pd.read_csv("OpenVirus.csv")

In [14]:
df_train.head()

,ArticleId,Text,Category
0,D:\try1\true\PMC6375641,west nile virus wnv has been circulating in ...,True
1,D:\try1\true\PMC6375641,west nile virus wnv is the cause of the larg...,True
2,D:\try1\true\PMC6410223,mosquito borne diseases constitute a large por...,True
3,D:\try1\true\PMC6445942,since acute flaccid myelitis afm a lo...,True
4,D:\try1\true\PMC6445942,since acute flaccid myelitis afm a lo...,True


In [15]:
#factorizing the categories into 0 and 1
df_train['category_id'] = df_train['Category'].factorize()[0]

In [16]:
df_train.groupby('Category').category_id.count().plot.bar(ylim=0)

In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(sublinear_tf=True, min_df=5, norm='l2', encoding='latin-1', ngram_range=(1, 2), stop_words='english')
features = tfidf.fit_transform(df_train.Text).toarray()
labels = df_train.category_id

In [18]:
category_to_id = {'true':0, 'false':1}
id_to_category = {0: 'true', 1: 'fasle'}